# Description
Calculate the proportion of checklists with at least one observation of a given species for each grid cell and year.

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta
import math
import matplotlib.pyplot as plt
import time

In [2]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/notebooks/calculate_arrival_date'

In [3]:
path = '/Users/alvastrand/Documents/OU/Research/data/'
os.chdir(path)

In [4]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/data'

In [5]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)

In [6]:
start_date = '0101'
end_date = '0731'
month = 'Apr'
year = '2020'
countries_states = 'US_states_east_Mississippi'

In [7]:
subdir = 'output/'
filename = "obligate_aerial_insectivores_ebird_species_codes.csv"

df = pd.read_csv(subdir + filename)

print('len(df) =', len(df))

len(df) = 19


In [8]:
df.head(2)

,scientific_name,common_name,species_code,category,taxon_order,order,family,report_as
0,Antrostomus arizonae,Mexican Whip-poor-will,souwpw1,species,3533,Caprimulgiformes,Caprimulgidae,NaN
1,Antrostomus carolinensis,Chuck-will's-widow,chwwid,species,3510,Caprimulgiformes,Caprimulgidae,NaN


In [9]:
for i in range(len(df)):
      
    print(i, df['species_code'].iloc[i], df['common_name'].iloc[i])

0 souwpw1 Mexican Whip-poor-will
1 chwwid Chuck-will's-widow
2 bucnig Buff-collared Nightjar
3 whip-p1 Eastern Whip-poor-will
4 lesnig Lesser Nighthawk
5 comnig Common Nighthawk
6 compoo Common Poorwill
7 whtswi White-throated Swift
8 chiswi Chimney Swift
9 vauswi Vaux's Swift
10 blkswi Black Swift
11 barswa Barn Swallow
12 cavswa Cave Swallow
13 cliswa Cliff Swallow
14 purmar Purple Martin
15 banswa Bank Swallow
16 nrwswa Northern Rough-winged Swallow
17 treswa Tree Swallow
18 vigswa Violet-green Swallow


In [10]:
def name_of_function(species, start_date, end_date, month, year, random_state, *args):
    
    subdir = 'eBird/ebd_output/'
    
    if args != ():
        
        countries_states = args[0]
        
        filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
        '_complete_zerofilled_grid_cells_rel' + month + '-' + year + '.csv'
        print('filename =', filename)
    
    df = pd.read_csv(subdir + filename)
    print('df.shape =', df.shape)
    
    df = df[['checklist_id', 'observation_date', 'species_observed', 'grid_cell']]
    
    df['species_observed_binary_values'] = np.where(df['species_observed'] == True, 1.0, 0.0)
    df['year'] = df['observation_date'].astype(str).str[:4]
    df['year'] = pd.to_numeric(df['year'])
    
    df = df[['grid_cell', 'year', 'checklist_id', 'observation_date', 'species_observed_binary_values']]
    
    # Filter

    df = df[df['year'] != 2020]
    print('len(df) =', len(df))
    
#     first_year = 2002
    first_year = 2003
#     first_year = 2004

#     first_year = 2010
#     first_year = 2012
#     first_year = 2009
    
#     first_year = 2015
#     first_year = 2014

    for start_year in range(first_year, first_year + 1):
        
        print('start_year =', start_year)
        
        # Filter

        df = df[df['year'] >= start_year]
        print('len(df) =', len(df))
        
#         grid_cell = 120
#         grid_cell = 128
#         grid_cell = 136
            
#         grid_cell = 117
#         grid_cell = 147
#         grid_cell = 151

        # Filter
        
#         df = df[df['grid_cell'] == grid_cell]
#         print('len(df) =', len(df))

        df_cnt = df[['checklist_id', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).count()
        df_cnt = df_cnt.rename(columns={'checklist_id': 'nb_checklists'})
        df_cnt = df_cnt.reset_index()
        print('len(df_cnt) =', len(df_cnt))
        print("len(df_cnt['grid_cell'].unique()) =", len(df_cnt['grid_cell'].unique()))

        start_month = 1
        start_day = 1

        end_month = 6
        # ?
        end_day = 30
        # ?
        
        end_year = 2019

        start_date_dt = date(end_year, start_month, start_day)
#         print('start_date_dt =', start_date_dt)

        end_date_dt = date(end_year, end_month, end_day)
#         print('end_date_dt =', end_date_dt)
        
        cnt_days = (end_date_dt - start_date_dt).days + 1
        print('cnt_days =', cnt_days)
        
        julian_days = list(range(1, cnt_days + 1)) * len(range(start_year, end_year + 1))
        assert(len(julian_days) == cnt_days * len(range(start_year, end_year + 1)))
#         print('julian_days[:2] =', julian_days[:2])
        
        dates = [pd.date_range(str(i) + '-01-01', periods=cnt_days) for i in range(start_year, end_year + 1)]
        assert(len(dates) == len(range(start_year, end_year + 1)))
#         print('dates[0][:2] =', dates[0][:2])
        
        dates = [list(date.strftime('%Y-%m-%d')) for date in dates]
        assert(len(dates) == len(range(start_year, end_year + 1)))
#         print('dates[0][:2] =', dates[0][:2])
#         print([date[-1] for date in dates])
        
        dates = [i for sublist in dates for i in sublist]
        assert(len(dates) == cnt_days * len(range(start_year, end_year + 1)))
#         print('dates[:2] =', dates[:2])
        
        df_julian_days_dates = pd.DataFrame({'julian_day': julian_days, 'observation_date': dates})
        df_julian_days_dates = df_julian_days_dates.sort_values(by=['julian_day', 'observation_date'])
        df_julian_days_dates = df_julian_days_dates.reset_index(drop=True)
        assert(len(df_julian_days_dates) == cnt_days * len(range(start_year, end_year + 1)))
        
        df_cnt = df_cnt.merge(df_julian_days_dates, on='observation_date')
        df_cnt = df_cnt[['grid_cell', 'julian_day', 'observation_date', 'nb_checklists']]
        df_cnt = df_cnt.sort_values(by=['grid_cell', 'julian_day', 'observation_date'])
        df_cnt = df_cnt.reset_index(drop=True)
        
        list_grid_cells_repeated = list(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates)
        list_grid_cells_repeated = sorted(list_grid_cells_repeated)
        assert(len(list_grid_cells_repeated) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
#         print('list_grid_cells_repeated[:2] =', list_grid_cells_repeated[:2])
        
        list_julian_days_repeated = list(df_julian_days_dates['julian_day']) * len(df_cnt['grid_cell'].unique())
        assert(len(list_julian_days_repeated) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
#         print('list_julian_days_repeated[:2] =', list_julian_days_repeated[:2])
        
        list_dates_repeated = list(df_julian_days_dates['observation_date']) * len(df_cnt['grid_cell'].unique())
        assert(len(list_dates_repeated) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
#         print('list_dates_repeated[:2] =', list_dates_repeated[:2])
        
        df_grid_cells_days_dates = pd.DataFrame({'grid_cell': list_grid_cells_repeated, 
                                                 'julian_day': list_julian_days_repeated, 
                                                 'observation_date': list_dates_repeated})
        assert(len(df_grid_cells_days_dates) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
        
        df_cnt_grid_cells_days_dates = df_grid_cells_days_dates.merge(
            df_cnt, how='left', on=['grid_cell', 'julian_day', 'observation_date'])
        df_cnt_grid_cells_days_dates['nb_checklists'] = df_cnt_grid_cells_days_dates['nb_checklists'].fillna(0)
        df_cnt_grid_cells_days_dates['nb_checklists'] = df_cnt_grid_cells_days_dates['nb_checklists'].astype(int)
        assert(len(df_cnt_grid_cells_days_dates) == len(df_grid_cells_days_dates))
        
        df_min_cnt = df_cnt_grid_cells_days_dates[['grid_cell', 'julian_day', 'nb_checklists']].groupby(
            ['grid_cell', 'julian_day']).min()
        df_min_cnt = df_min_cnt.rename(columns={'nb_checklists': 'min_nb_checklists'})
        df_min_cnt = df_min_cnt.reset_index()
        assert(len(df_min_cnt) == len(df_cnt['grid_cell'].unique()) * cnt_days)
        
        df_cnt_zero = df_min_cnt[['grid_cell', 'min_nb_checklists']].groupby(
            ['grid_cell']).agg(lambda x: x.eq(0).sum())
        df_cnt_zero = df_cnt_zero.rename(columns={'min_nb_checklists': 'cnt_zero'})
        df_cnt_zero = df_cnt_zero.reset_index()
        assert(len(df_cnt_zero) == len(df_cnt['grid_cell'].unique()))
        
        threshold = cnt_days/2
        
        # Filter
        
        list_grid_cells = list(df_cnt_zero[df_cnt_zero['cnt_zero'] < threshold]['grid_cell'])
        print('len(list_grid_cells):', len(list_grid_cells))
        
        print("len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]) =", 
              len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]))
        
        
        df_grid_cells_threshold = df[df['grid_cell'].isin(list_grid_cells)]
        print('len(df_grid_cells_threshold):', len(df_grid_cells_threshold))

        # Calculate the number of checklists for each grid cell and date
        df_cnt_not_sampled = df_grid_cells_threshold[['checklist_id', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).count()
        df_cnt_not_sampled = df_cnt_not_sampled.rename(columns={'checklist_id': 'nb_checklists'})
        print(len(df_cnt_not_sampled))

        # Calculate the number of checklists with at least one observation of the species of interest for each grid 
        # cell and date
        df_sum_not_sampled = df_grid_cells_threshold[
            ['species_observed_binary_values', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).sum()
        df_sum_not_sampled = df_sum_not_sampled.rename(
            columns={'species_observed_binary_values': 'nb_checklists_species'})
        assert(len(df_sum_not_sampled) == len(df_cnt_not_sampled))

        df_grp_not_sampled = df_cnt_not_sampled.merge(df_sum_not_sampled, left_index=True, right_index=True)
        # Calculate the proportion of checklists with at least one observation of the species of interest for each 
        # grid cell and date
        df_grp_not_sampled['prop'] = df_grp_not_sampled['nb_checklists_species']/df_grp_not_sampled['nb_checklists']
        df_grp_not_sampled = df_grp_not_sampled.reset_index()
        df_grp_not_sampled['year'] = df_grp_not_sampled['observation_date'].astype(str).str[:4]
        assert(len(df_grp_not_sampled) == len(df_cnt_not_sampled))
        
        df_grid_cells_days_dates_years = df_grid_cells_days_dates[
            df_grid_cells_days_dates['grid_cell'].isin(list_grid_cells)]
        df_grid_cells_days_dates_years = df_grid_cells_days_dates_years.reset_index(drop=True)
        df_grid_cells_days_dates_years['year'] = df_grid_cells_days_dates_years[
            'observation_date'].astype(str).str[:4]
        df_grid_cells_days_dates_years = df_grid_cells_days_dates_years[
            ['grid_cell', 'year', 'julian_day', 'observation_date']]
        assert(len(df_grid_cells_days_dates_years) == 
               len(list_grid_cells) * len(range(start_year, end_year + 1)) * cnt_days)
        
        df_grp_not_sampled_dates = df_grid_cells_days_dates_years.merge(
            df_grp_not_sampled, how='left', on=['grid_cell', 'observation_date', 'year'])
        df_grp_not_sampled_dates = df_grp_not_sampled_dates[
            ['grid_cell', 'year', 'julian_day', 'observation_date', 'nb_checklists', 'nb_checklists_species', 
             'prop']]
        df_grp_not_sampled_dates = df_grp_not_sampled_dates.sort_values(by=['grid_cell', 'year', 'julian_day'])
        df_grp_not_sampled_dates = df_grp_not_sampled_dates.reset_index(drop=True)
        
        # Perform arcsine transformation on the proportions
        df_grp_not_sampled_dates['prop_arcsine'] = df_grp_not_sampled_dates['prop'].apply(
            lambda x: np.arcsin(math.sqrt(x)))
        # ?
        
        assert(len(df_grp_not_sampled_dates) == len(df_grid_cells_days_dates_years))
        
        df_sum_prop_not_sampled = df_grp_not_sampled_dates[['grid_cell', 'year', 'prop']].groupby(
            ['grid_cell', 'year']).sum()
        df_sum_prop_not_sampled = df_sum_prop_not_sampled.reset_index()
        df_sum_prop_not_sampled = df_sum_prop_not_sampled.rename(columns={'prop': 'sum_prop'})
        assert(len(df_sum_prop_not_sampled) == len(list_grid_cells) * len(range(start_year, end_year + 1)))
        
        skip_species = 0
        
        if len(df_sum_prop_not_sampled[df_sum_prop_not_sampled['sum_prop'] == 0]) == len(df_sum_prop_not_sampled):
            
            skip_species = 1
            print('Skip species with species code ' + species + '.')
            
            break
        
        if len(df_sum_prop_not_sampled[df_sum_prop_not_sampled['sum_prop'] == 0]) > 0:
        
            df_sum_prop_zero_not_sampled = df_sum_prop_not_sampled[
                df_sum_prop_not_sampled['sum_prop'] == 0][['grid_cell', 'year']]
            print('len(df_sum_prop_zero_not_sampled) =', len(df_sum_prop_zero_not_sampled))
            print('len(df_sum_prop_not_sampled) - len(df_sum_prop_zero_not_sampled) =', 
                  len(df_sum_prop_not_sampled) - len(df_sum_prop_zero_not_sampled))
            
#             print(df_sum_prop_zero_not_sampled.head(2))
            
            df_grp_not_sampled_dates = df_grp_not_sampled_dates.merge(
                df_sum_prop_zero_not_sampled, how='left', on=['grid_cell', 'year'], indicator=True)
            assert(len(df_grp_not_sampled_dates) == len(df_grid_cells_days_dates_years))
            
            df_grp_not_sampled_dates = df_grp_not_sampled_dates[df_grp_not_sampled_dates['_merge'] == 'left_only']
            print('len(df_grp_not_sampled_dates) =', len(df_grp_not_sampled_dates))
            
            df_grp_not_sampled_dates = df_grp_not_sampled_dates.drop(columns=['_merge'])
                        
        df_sampled = pd.DataFrame()

        for i in range(len(list_grid_cells)):

#             i = 0

            print('i =', i)

            grid_cell_threshold = list_grid_cells[i]

            df_grid_cell = df[df['grid_cell'] == grid_cell_threshold]
            print('len(df_grid_cell) =', len(df_grid_cell))

            df_min_cnt_grid_cell = df_min_cnt[df_min_cnt['grid_cell'] == grid_cell_threshold]
            df_min_cnt_grid_cell = df_min_cnt_grid_cell.reset_index(drop=True)
            assert(len(df_min_cnt_grid_cell) == cnt_days)

            df_min_cnt_greater_than_grid_cell = df_min_cnt_grid_cell[df_min_cnt_grid_cell['min_nb_checklists'] > 0]
            df_min_cnt_greater_than_grid_cell = df_min_cnt_greater_than_grid_cell.reset_index(drop=True)
            print('len(df_min_cnt_greater_than_grid_cell) =', len(df_min_cnt_greater_than_grid_cell))

            # observation_date
            df_min_cnt_greater_than_dates_grid_cell = df_julian_days_dates.merge(
                df_min_cnt_greater_than_grid_cell, on='julian_day')
            assert(len(df_min_cnt_greater_than_dates_grid_cell) == 
                   len(df_min_cnt_greater_than_grid_cell) * len(range(start_year, end_year + 1)))

            list_dates_grid_cell = list(df_min_cnt_greater_than_dates_grid_cell['observation_date'])
            assert(len(list_dates_grid_cell) == len(df_min_cnt_greater_than_dates_grid_cell))

            list_min_cnt_grid_cell = list(df_min_cnt_greater_than_dates_grid_cell['min_nb_checklists'])
            assert(len(list_min_cnt_grid_cell) == len(df_min_cnt_greater_than_dates_grid_cell))
            
            for j in range(len(list_dates_grid_cell)):
                
#                 j = 0

                df_grid_cell_date = df_grid_cell[df_grid_cell['observation_date'] == list_dates_grid_cell[j]]

                df_grid_cell_date = df_grid_cell_date.sample(list_min_cnt_grid_cell[j], random_state=random_state)
#                 assert(len(df_grid_cell_date) == list_min_cnt_grid_cell[j])

                df_sampled = df_sampled.append(df_grid_cell_date)
#                 assert(len(df_sampled) == len(df_grid_cell_date))
            
        print('len(df_sampled):', len(df_sampled))
                
        # Calculate the number of checklists for each grid cell and date
        df_cnt_sampled = df_sampled[['checklist_id', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).count()
        df_cnt_sampled = df_cnt_sampled.rename(columns={'checklist_id': 'nb_checklists'})
        print(len(df_cnt_sampled))

        # Calculate the number of checklists with at least one observation of the species of interest for each grid 
        # cell and date
        df_sum_sampled = df_sampled[['species_observed_binary_values', 'grid_cell', 'observation_date']].groupby([
            'grid_cell', 'observation_date']).sum()
        df_sum_sampled = df_sum_sampled.rename(columns={'species_observed_binary_values': 'nb_checklists_species'})
        assert(len(df_sum_sampled) == len(df_cnt_sampled))

        df_grp_sampled = df_cnt_sampled.merge(df_sum_sampled, left_index=True, right_index=True)
        # Calculate the proportion of checklists with at least one observation of the species of interest for each 
        # grid cell and date
        df_grp_sampled['prop'] = df_grp_sampled['nb_checklists_species']/df_grp_sampled['nb_checklists']
        df_grp_sampled = df_grp_sampled.reset_index()
        df_grp_sampled['year'] = df_grp_sampled['observation_date'].astype(str).str[:4]
        assert(len(df_grp_sampled) == len(df_cnt_sampled))
        
        df_grp_sampled_dates = df_grid_cells_days_dates_years.merge(
            df_grp_sampled, how='left', on=['grid_cell', 'observation_date', 'year'])
        df_grp_sampled_dates = df_grp_sampled_dates[
            ['grid_cell', 'year', 'julian_day', 'observation_date', 'nb_checklists', 'nb_checklists_species', 
             'prop']]
        df_grp_sampled_dates = df_grp_sampled_dates.sort_values(by=['grid_cell', 'year', 'julian_day'])
        df_grp_sampled_dates = df_grp_sampled_dates.reset_index(drop=True)
        
        # Perform arcsine transformation on the proportions
        df_grp_sampled_dates['prop_arcsine'] = df_grp_sampled_dates['prop'].apply(lambda x: np.arcsin(math.sqrt(x)))
        # ?
        
        assert(len(df_grp_sampled_dates) == len(df_grid_cells_days_dates_years))
        
        df_sum_prop_sampled = df_grp_sampled_dates[['grid_cell', 'year', 'prop']].groupby(
            ['grid_cell', 'year']).sum()
        df_sum_prop_sampled = df_sum_prop_sampled.reset_index()
        df_sum_prop_sampled = df_sum_prop_sampled.rename(columns={'prop': 'sum_prop'})
        assert(len(df_sum_prop_sampled) == len(list_grid_cells) * len(range(start_year, end_year + 1)))
        
        # If there are grid cells and years that only have proportions that are equal to zero
        if len(df_sum_prop_sampled[df_sum_prop_sampled['sum_prop'] == 0]) > 0:
        
            df_sum_prop_zero_sampled = df_sum_prop_sampled[
                df_sum_prop_sampled['sum_prop'] == 0][['grid_cell', 'year']]
            print('len(df_sum_prop_zero_sampled) =', len(df_sum_prop_zero_sampled))
            print('len(df_sum_prop_sampled) - len(df_sum_prop_zero_sampled) =', 
                  len(df_sum_prop_sampled) - len(df_sum_prop_zero_sampled))
            
#             print(df_sum_prop_zero_sampled.head(2))
            
            df_grp_sampled_dates = df_grp_sampled_dates.merge(
                df_sum_prop_zero_sampled, how='left', on=['grid_cell', 'year'], indicator=True)
            assert(len(df_grp_sampled_dates) == len(df_grid_cells_days_dates_years))
            
            df_grp_sampled_dates = df_grp_sampled_dates[df_grp_sampled_dates['_merge'] == 'left_only']
            print('len(df_grp_sampled_dates) =', len(df_grp_sampled_dates))
            
            df_grp_sampled_dates = df_grp_sampled_dates.drop(columns=['_merge'])
        
        # If there's at least one grid cell and year that has at least one proportion that's not equal to zero
        if (len(df_sum_prop_sampled[df_sum_prop_sampled['sum_prop'] == 0]) < len(df_sum_prop_sampled)):
    
            if args != ():

                countries_states = args[0]

                filename_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
                end_date + '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + \
                str(end_year) + '_sampled_' + str(random_state) + '_rel' + month + '-' + year + '.csv'

#                 filename_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
#                 end_date + '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_' + \
#                 str(start_year) + '_' + str(end_year) + '_sampled_' + str(random_state) + '_rel' + month + '-' + \
#                 year + '.csv'

                filename_not_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
                end_date + '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + \
                str(end_year) + '_not_sampled_rel' + month + '-' + year + '.csv'

#                 filename_not_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
#                 end_date + '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_' + \
#                 str(start_year) + '_' + str(end_year) + '_not_sampled_rel' + month + '-' + year + '.csv'

                print(filename_sampled)
                print(filename_not_sampled)

            df_grp_sampled_dates.to_csv(subdir + filename_sampled, index=False)
            df_grp_not_sampled_dates.to_csv(subdir + filename_not_sampled, index=False)

    return df, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, df_min_cnt, \
df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, df_grp_not_sampled, \
df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, df_sum_prop_sampled, \
df_sum_prop_not_sampled, skip_species

In [11]:
# species_cnt = 0

# for i in range(len(df)):
    
#     print(i)
  
#     species = df['species_code'].iloc[i]
#     print(species)
    
#     if ((species == 'souwpw1') | (species == 'bucnig') | (species == 'compoo') | (species == 'whtswi') | 
#         (species == 'blkswi') | (species == 'treswa') | (species == 'barswa') | (species == 'chwwid') | 
#         (species == 'whip-p1') | (species == 'lesnig') | (species == 'comnig') | (species == 'chiswi') | 
#         (species == 'vauswi')):
#         continue
    
#     for random_state in range(1, 6):

#         df_grid_cells, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, \
#         df_min_cnt, df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
#         df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
#         df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, \
#         df_grp_not_sampled, df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, \
#         df_sum_prop_sampled, df_sum_prop_not_sampled, skip_species = name_of_function(
#             species, start_date, end_date, month, year, random_state, countries_states)
        
#         if skip_species == 1:
#             break
    
#     species_cnt += 1
    
# print(species_cnt)

In [12]:
# species = 'barswa'
species = 'treswa'

random_state = 1

start_time = time.time()

df_grid_cells, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, \
df_min_cnt, df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, \
df_grp_not_sampled, df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, \
df_sum_prop_sampled, df_sum_prop_not_sampled, skip_species = name_of_function(
    species, start_date, end_date, month, year, random_state, countries_states)

# for random_state in range(1, 6):

#     df_grid_cells, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, \
#     df_min_cnt, df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
#     df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
#     df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, \
#     df_grp_not_sampled, df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, \
#     df_sum_prop_sampled, df_sum_prop_not_sampled, skip_species = name_of_function(
#         species, start_date, end_date, month, year, random_state, countries_states)

print('(time.time() - start_time)/60 =', (time.time() - start_time)/60)

filename = ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_relApr-2020.csv


/Users/alvastrand/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (11,29) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


df.shape = (8351771, 35)
len(df) = 7227884
start_year = 2003
len(df) = 7148340
len(df_cnt) = 226213
len(df_cnt['grid_cell'].unique()) = 108
cnt_days = 181
len(list_grid_cells): 6
len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]) = 34
len(df_grid_cells_threshold): 2038893
21497
i = 0
len(df_grid_cell) = 149274
len(df_min_cnt_greater_than_grid_cell) = 144
i = 1
len(df_grid_cell) = 479075
len(df_min_cnt_greater_than_grid_cell) = 162
i = 2
len(df_grid_cell) = 407506
len(df_min_cnt_greater_than_grid_cell) = 173
i = 3
len(df_grid_cell) = 330255
len(df_min_cnt_greater_than_grid_cell) = 157
i = 4
len(df_grid_cell) = 231351
len(df_min_cnt_greater_than_grid_cell) = 177
i = 5
len(df_grid_cell) = 441432
len(df_min_cnt_greater_than_grid_cell) = 152
len(df_sampled): 45186
16405
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2003_2019_sampled_1_relApr-2020.csv
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_

In [13]:
df_grid_cells.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
1,131,2003,S1018111,2003-01-04,0.0
2,128,2003,S1018363,2003-01-05,0.0


In [14]:
df_julian_days_dates.head(2)

,julian_day,observation_date
0,1,2003-01-01
1,1,2004-01-01


In [15]:
df_cnt.head(2)

,grid_cell,julian_day,observation_date,nb_checklists
0,4,26,2014-01-26,1
1,4,33,2013-02-02,1


In [16]:
df_grid_cells_days_dates.head(2)

,grid_cell,julian_day,observation_date
0,4,1,2003-01-01
1,4,1,2004-01-01


In [17]:
df_cnt_grid_cells_days_dates.head(2)

,grid_cell,julian_day,observation_date,nb_checklists
0,4,1,2003-01-01,0
1,4,1,2004-01-01,0


In [18]:
df_min_cnt.head(2)

,grid_cell,julian_day,min_nb_checklists
0,4,1,0
1,4,2,0


In [19]:
df_cnt_zero.head(2)

,grid_cell,cnt_zero
0,4,181
1,5,181


In [20]:
df_grid_cell.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
10,136,2003,S1040282,2003-01-18,0.0
85,136,2003,S1196445,2003-06-08,0.0


In [21]:
df_min_cnt_grid_cell.head(2)

,grid_cell,julian_day,min_nb_checklists
0,136,1,7
1,136,2,1


In [22]:
df_min_cnt_greater_than_grid_cell.head(2)

,grid_cell,julian_day,min_nb_checklists
0,136,1,7
1,136,2,1


In [23]:
df_min_cnt_greater_than_dates_grid_cell.head(2)

,julian_day,observation_date,grid_cell,min_nb_checklists
0,1,2003-01-01,136,7
1,1,2004-01-01,136,7


In [24]:
df_grid_cell_date.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
7481417,136,2019,S57809309,2019-06-30,1.0
1906874,136,2019,S57808964,2019-06-30,0.0


In [25]:
df_sampled.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
5901205,74,2003,S20350109,2003-01-01,0.0
357582,74,2004,S1528487,2004-01-01,0.0


In [26]:
# df_cnt_sampled.head(2)

In [27]:
# df_sum_sampled.head(2)

In [28]:
df_grp_sampled.head(2)

,grid_cell,observation_date,nb_checklists,nb_checklists_species,prop,year
0,74,2003-01-01,1,0.0,0.0,2003
1,74,2003-01-03,1,0.0,0.0,2003


In [29]:
df_grid_cells_threshold.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
2,128,2003,S1018363,2003-01-05,0.0
9,128,2003,S1038802,2003-02-02,0.0


In [30]:
# df_cnt_not_sampled.head(2)

In [31]:
# df_sum_not_sampled.head(2)

In [32]:
df_grp_not_sampled.tail(2)

,grid_cell,observation_date,nb_checklists,nb_checklists_species,prop,year
21495,136,2019-07-30,172,26.0,0.151163,2019
21496,136,2019-07-31,156,22.0,0.141026,2019


In [33]:
df_grid_cells_days_dates_years.head(2)

,grid_cell,year,julian_day,observation_date
0,74,2003,1,2003-01-01
1,74,2004,1,2004-01-01


In [34]:
df_grp_sampled_dates.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine
0,74,2003,1,2003-01-01,1.0,0.0,0.0,0.0
1,74,2003,2,2003-01-02,NaN,NaN,NaN,NaN


In [35]:
df_grp_not_sampled_dates.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine
0,74,2003,1,2003-01-01,2.0,0.0,0.0,0.0
1,74,2003,2,2003-01-02,NaN,NaN,NaN,NaN


In [36]:
df_sum_prop_sampled.head(2)

,grid_cell,year,sum_prop
0,74,2003,2.5
1,74,2004,2.5


In [37]:
df_sum_prop_not_sampled.head(2)

,grid_cell,year,sum_prop
0,74,2003,4.566667
1,74,2004,2.528571


In [ ]:
# df_grp_sampled['prop'].hist()
# plt.show()

In [ ]:
# df_grp_not_sampled['prop'].hist()
# plt.show()